In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [14]:
path = '../raw_data/train'

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=90,
    validation_split = 0.2)

In [16]:
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2)

In [17]:
train_generator = train_datagen.flow_from_directory(
    path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed = 123)

Found 3460 images belonging to 16 classes.


In [18]:
val_generator = valid_datagen.flow_from_directory(
    path, # same directory as training data
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    seed = 123)

Found 857 images belonging to 16 classes.


In [19]:
class_weight = {0:4. ,
                1:2., 
                2:1.5 ,
                3:2.,
                4:3.,
                5:2.,
                6:1.5,
                7:4.,
                8:2.,
                9:3.,
                10:1.,
                11:4.,
                12:3.,
                13:2.,
                14:3.,
                15:2.
               }

In [20]:
vgg16_model = VGG16(include_top=False, input_shape=(256,256,3))

model=Sequential()
for layers in vgg16_model.layers:
    model.add(layers)
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dense(4096,activation='relu'))
for layer in model.layers:
    layer.trainable=False
model.add(Dense(16,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)      

In [ ]:
# # Set the first layers to be untrainable
# model.trainable = False
# # Add layers to the mdoel
# flatten_layer = layers.Flatten()
# dense_layer = layers.Dense(100, activation='relu')
# prediction_layer = layers.Dense(16, activation='softmax')

# model = Sequential([model,
#                     flatten_layer,
#                     dense_layer,
#                     prediction_layer])
# model.summary()

In [24]:
# model.compile(optimizer=optimizers.SGD(learning_rate = 0.1),
#           loss=SparseCategoricalCrossentropy(from_logits=True),
#           metrics=['accuracy'])

In [25]:
optimizer=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.0)

In [32]:
model.compile(optimizer,loss=SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [33]:
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [22]:
# callbacks = EarlyStopping(
#     monitor='val_loss',
#     patience=3,
#     mode='min')

In [34]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = val_generator,
    validation_steps = val_generator.samples // 32,
    epochs = 20,
    callbacks = [learning_rate_reduction])

Epoch 1/10
108/108 [==============================] - 914s 8s/step - loss: 2.7057 - accuracy: 0.1628 - val_loss: 2.6780 - val_accuracy: 0.2103
Epoch 2/10
108/108 [==============================] - 855s 8s/step - loss: 2.6515 - accuracy: 0.2293 - val_loss: 2.6282 - val_accuracy: 0.2464
Epoch 3/10
108/108 [==============================] - 879s 8s/step - loss: 2.6194 - accuracy: 0.2660 - val_loss: 2.6034 - val_accuracy: 0.2897
Epoch 4/10
108/108 [==============================] - 845s 8s/step - loss: 2.5975 - accuracy: 0.2928 - val_loss: 2.5980 - val_accuracy: 0.2945
Epoch 5/10
108/108 [==============================] - 834s 8s/step - loss: 2.5777 - accuracy: 0.3168 - val_loss: 2.5903 - val_accuracy: 0.2849
Epoch 6/10
108/108 [==============================] - 863s 8s/step - loss: 2.5709 - accuracy: 0.3139 - val_loss: 2.5659 - val_accuracy: 0.3245
Epoch 7/10
108/108 [==============================] - 896s 8s/step - loss: 2.5612 - accuracy: 0.3296 - val_loss: 2.5454 - val_accuracy: 0.3413

In [35]:
model.save(r'../model/roxane_vgg_1')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../model/roxane_vgg_1/assets
